# Выбор локации для скважины

Добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину. Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищем месторождения, для каждого определяем значения признаков;
- Строим модель и оценивают объём запасов;
- Выбираем месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль будует равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score 
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Загрузим и подготовим данные по каждому региону в отдельности. 

In [3]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропусков нет, с типами данных всё в порядке. Проверим, нет ли скважин-дубликатов. 

In [5]:
data_1['id'].value_counts().head(20)

A5aEY    2
QcMuo    2
AGS9W    2
fiKDv    2
TtcGQ    2
bxg6G    2
bsk9y    2
74z30    2
Tdehs    2
HZww2    2
F5Cub    1
MGoBr    1
jNIfK    1
QCHkh    1
HGZuB    1
JGZEn    1
jhIDn    1
AJWJJ    1
oK8iW    1
Xbd8H    1
Name: id, dtype: int64

Мы нашли скважины, которые фигурируют в данных дважды. Посмотрим, насколько различаются у них признаки. Проверим несколько скважин. 

In [6]:
data_1.query('id == "bxg6G"')

,id,f0,f1,f2,product
1364,bxg6G,0.411645,0.856830,-3.653440,73.604260
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798


In [7]:
data_1.query('id == "QcMuo"')

,id,f0,f1,f2,product
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675


In [8]:
data_1.query('id == "HZww2"')

,id,f0,f1,f2,product
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695


Откуда могли взяться эти дубликаты. Возможно, это ошибка ввода данных. Или это разные замеры. 
В любом случае, так как признаки и целевой признак в строках с одним id могут сильно отличаться, скорее всего, это потенциально неверные данные, которые могут ввести в заблуждение модель. Поэтому, так как строк с дублирующимися id всего 20 (0,02% от данных), то их следует удалить.  

In [9]:
data_1 = data_1.drop_duplicates( "id" , keep=False)

Проверим, удалось ли избавиться от дубликатов

In [10]:
data_1['id'].value_counts().head(20)

T1O5L    1
ZS5aA    1
KG5iq    1
zVWTj    1
CQ6LU    1
MGoBr    1
jNIfK    1
QCHkh    1
JGZEn    1
jhIDn    1
AJWJJ    1
oK8iW    1
mPPpY    1
QcNAG    1
9GTLP    1
84VQ4    1
STU6d    1
5aUbq    1
psNh7    1
Xbd8H    1
Name: id, dtype: int64

In [11]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99980 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99980 non-null  object 
 1   f0       99980 non-null  float64
 2   f1       99980 non-null  float64
 3   f2       99980 non-null  float64
 4   product  99980 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


То же проделаем с данными по двум другим регионам. 

In [12]:
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [13]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [14]:
data_2['id'].value_counts().head(20)

LHZR0    2
bfPNe    2
wt4Uk    2
5ltQ6    2
6KaYB    1
prWru    1
y9FT0    1
lkLRx    1
ReOAT    1
xj46q    1
9lvJF    1
teekR    1
uU3mh    1
A8qja    1
11TyV    1
JNSKz    1
OmPZz    1
HtGWy    1
LVWoU    1
iAYCL    1
Name: id, dtype: int64

В данных по второму региону строк с дублирующимися id ещё меньше (0,008%), удалим их. 

In [15]:
data_2 = data_2.drop_duplicates( "id" , keep=False)
print(data_2['id'].value_counts().head(10))
print(data_2.info())

A8qja    1
oe2u1    1
JNSKz    1
dfVGv    1
prWru    1
y9FT0    1
lkLRx    1
ReOAT    1
xj46q    1
9lvJF    1
Name: id, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99992 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99992 non-null  object 
 1   f0       99992 non-null  float64
 2   f1       99992 non-null  float64
 3   f2       99992 non-null  float64
 4   product  99992 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB
None


Теперь обратимся к третьему региону

In [16]:
data_3 = pd.read_csv('/datasets/geo_data_2.csv')
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [17]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [18]:
data_3['id'].value_counts().head(20)

VF7Jo    2
xCHr8    2
KUPhW    2
Vcm5J    2
aGZOU    1
1MNJ6    1
DmE46    1
8Ntut    1
2VpWD    1
KOsfN    1
wSxfD    1
Ij9KZ    1
RuFkW    1
WAiwq    1
Hp7KJ    1
XyV4o    1
HDtcs    1
R7YRl    1
Q4x7M    1
mOMJW    1
Name: id, dtype: int64

Скважин-дубликатов столько же, сколько было и в данных для второго региона. Удалим их и проверим, получилось ли. 

In [19]:
data_3 = data_3.drop_duplicates( "id" , keep=False)
print(data_3['id'].value_counts().head(10))
print(data_3.info())

Hp7KJ    1
M6xRz    1
8Ntut    1
2VpWD    1
KOsfN    1
wSxfD    1
Ij9KZ    1
RuFkW    1
WAiwq    1
aGZOU    1
Name: id, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99992 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99992 non-null  object 
 1   f0       99992 non-null  float64
 2   f1       99992 non-null  float64
 3   f2       99992 non-null  float64
 4   product  99992 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB
None


Столбец id не нужен для обучения, поэтому удалим его и посмотрим, удалось ли. 

In [20]:
data_1 = data_1.drop('id', axis=1)
data_2 = data_2.drop('id', axis=1)
data_3 = data_3.drop('id', axis=1)
print(data_1.columns)
print(data_2.columns)
print(data_3.columns)

Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Index(['f0', 'f1', 'f2', 'product'], dtype='object')


Данные загружены и были заренее подготовлены, масштабирование не требуется, поэтому можно переходить к разбиению данных на выборки. 

## Обучение и проверка модели

Теперь разобьём данные на обучающую и валидационную выборки в соотношении 75:25

In [21]:
features_1 = data_1.drop(['product'], axis=1)
target_1 = data_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1,
                                                                                      test_size=0.25,
                                                                                      random_state=12345)


In [22]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predicted_valid_1 = model_1.predict(features_valid_1)
rmse_1 = (mean_squared_error(target_valid_1, predicted_valid_1))**0.5
R2_1 = r2_score(target_valid_1, predicted_valid_1)
print('регион 1:','\n', 'Средний запас сырья (предсказание) =', predicted_valid_1.mean(), 'тыс. баррелей', '\n',
      'Средний запас сырья (реальный) =', target_valid_1.mean(), 'тыс. баррелей', '\n', 'RMSE =', rmse_1, 'тыс. баррелей')
print()
 
# RMSE константной модели
predicted_valid_k = pd.Series(target_train_1.mean(), index=target_valid_1.index)
rmse_k = (mean_squared_error(target_valid_1, predicted_valid_k))**0.5
print('RMSE константной модели =', rmse_k )

регион 1: 
 Средний запас сырья (предсказание) = 92.42384109947359 тыс. баррелей 
 Средний запас сырья (реальный) = 92.39348967901073 тыс. баррелей 
 RMSE = 37.716904960382735 тыс. баррелей

RMSE константной модели = 44.34727648978901


Средний запас сырья по предсказаниям модели близок к реальному, RMSE выше, чем у константной модели.
Обучим модели для остальных регионов и исследуем результаты. 

In [23]:
#Обучаем модель для второго региона
features_2 = data_2.drop(['product'], axis=1)
target_2 = data_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2,
                                                                                      test_size=0.25,
                                                                                      random_state=12345)

In [24]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predicted_valid_2 = model_2.predict(features_valid_2)
rmse_2 = (mean_squared_error(target_valid_2, predicted_valid_2))**0.5
print('регион 2:','\n', 'Средний запас сырья (предсказание) =', predicted_valid_2.mean(), 'тыс. баррелей', '\n',
      'Средний запас сырья (реальный) =', target_valid_2.mean(), 'тыс. баррелей', '\n', 'RMSE =', rmse_2)
print()
 
# RMSE константной модели
predicted_valid_k2 = pd.Series(target_train_2.mean(), index=target_valid_2.index)
rmse_k2 = (mean_squared_error(target_valid_2, predicted_valid_k2))**0.5
print('RMSE константной модели =', rmse_k2 )

регион 2: 
 Средний запас сырья (предсказание) = 68.98311857983123 тыс. баррелей 
 Средний запас сырья (реальный) = 68.98037303230389 тыс. баррелей 
 RMSE = 0.8914901390348537

RMSE константной модели = 45.97003721244109


Во втором регионе RMSE гораздо выше, чем в константной модели и чем в первом регионе.

In [25]:
#Обучаем модель для третьего региона
features_3 = data_3.drop(['product'], axis=1)
target_3 = data_3['product']
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(features_3, target_3,
                                                                                      test_size=0.25,
                                                                                      random_state=12345)

In [26]:
model_3 = LinearRegression()
model_3.fit(features_train_3, target_train_3)
predicted_valid_3 = model_3.predict(features_valid_3)
rmse_3 = (mean_squared_error(target_valid_3, predicted_valid_3))**0.5
print('регион 3:','\n', 'Средний запас сырья (предсказание) =', predicted_valid_3.mean(), 'тыс. баррелей', '\n',
      'Средний запас сырья (реальный) =', target_valid_3.mean(), 'тыс. баррелей', '\n', 'RMSE =', rmse_3, 'тыс. баррелей')
print()
 
# RMSE константной модели
predicted_valid_k3 = pd.Series(target_train_3.mean(), index=target_valid_3.index)
rmse_k3 = (mean_squared_error(target_valid_3, predicted_valid_k3))**0.5
print('RMSE константной модели =', rmse_k3 )

регион 3: 
 Средний запас сырья (предсказание) = 95.11622302076478 тыс. баррелей 
 Средний запас сырья (реальный) = 94.54781801775303 тыс. баррелей 
 RMSE = 39.975543264382345 тыс. баррелей

RMSE константной модели = 44.57522707537966


Результаты обучения модели похожи на первый регион: модель предсказывает чуть лучше, чем случайная.

## Подготовка к расчёту прибыли

Сохраним в отдельных переменных все ключевые значения, необходимые для расчёта прибыли

In [27]:
#бюджет на разработку скважин в регионе
budget = 10000000000

#доход с каждой единицы продукта (тысяч рублей за тысячу баррелей)
income = 450000

#количество исследуемых точек при разведке региона
points_selected = 500

#количество выбираемых лучших точек для разработки
points_best = 200


Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Для этого сначала найдём бюджет для разработки одной скважины.

In [28]:
budget_point = budget / points_best
product_value_point = budget_point / income
print('Достаточный объем сырья для безубыточной разработки новой скважины равен', product_value_point, 'тыс. баррелей')
print()

Достаточный объем сырья для безубыточной разработки новой скважины равен 111.11111111111111 тыс. баррелей



Сравним полученный объём сырья со средним запасом в каждом регионе.

In [29]:
print('Средний запас сырья в регионе 1:', data_1['product'].mean())
print('Средний запас сырья в регионе 2:', data_2['product'].mean())
print('Средний запас сырья в регионе 3:', data_3['product'].mean())
print('Достаточный запас сырья:', product_value_point)

Средний запас сырья в регионе 1: 92.49916597893444
Средний запас сырья в регионе 2: 68.82414772665173
Средний запас сырья в регионе 3: 94.99876686768079
Достаточный запас сырья: 111.11111111111111


Достаточный запас сырья для безубыточной разработки новой скважины оказался значимо больше, чем средние запасы в каждом регионе. Это означает, что нам необходимо максимально тщательно подойти к выбору региона для новой разработки, чтобы новый проект был рентабельным. Есть вероятность, что бюджет на разработку не окупится. Поэтому далее мы должны использовать технику bootstrap, чтобы просчитать риски. 
Напишем функцию для расчета прибыли. 

In [30]:
#функция расчета прибыли
def profit(target, predicted):
    predicted = pd.Series(data = predicted, index = target.index)
    predicted_sorted = predicted.sort_values(ascending=False)
    selected = target[predicted_sorted.index][:points_best]
    return income * selected.sum() - budget


#target.values[pred_sort.index][:best_points]
#predicted_0 = pd.Series(data = predicted_valid_0, index = target_valid_0.index)

## Расчёт прибыли и рисков 

In [31]:
#Напишем функцию для процедуры Bootstrap для расчета рисков и прибыли


state = np.random.RandomState(12345)
 
def bootstrap(target, predicted):
    values = []
    for i in range(1000):
        target = pd.Series(target)
        target_subsample = target.sample(n=points_selected, replace=True, random_state=state)
        probs_subsample = pd.Series(data = predicted, index=target.index)[target_subsample.index]
        profit_boot = profit(target_subsample, probs_subsample)
        values.append(profit_boot)
 
    values = pd.Series(values)
 
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risk = (values < 0).mean()*100
 
    print("Средняя прибыль:", values.mean())
    print('95%-ый доверительный интервал:', lower, '-', upper)
    print('Риск убытков:', risk)
    


In [38]:
#Проведём процедуру bootstrap для каждого региона
bootstrap(target_valid_1, predicted_valid_1)

Средняя прибыль: 463640532.45456076
95%-ый доверительный интервал: -50829441.10239987 - 974589534.6230257
Риск убытков: 4.3999999999999995


In [37]:
bootstrap(target_valid_2, predicted_valid_2)

Средняя прибыль: 534777965.5867257
95%-ый доверительный интервал: 128762334.71869636 - 961538795.8060088
Риск убытков: 0.5


In [36]:
bootstrap(target_valid_3, predicted_valid_3)

Средняя прибыль: 350149184.42709255
95%-ый доверительный интервал: -217500697.77422526 - 876483747.1172203
Риск убытков: 10.6


Как видим, самый низкий риск убытков во втором регионе, к тому же только в нём нижний квантиль является положительным, значит, риск наименьший. Поэтому мы будем рекомендовать для разработки выбрать именно его. 

## Выводы

Мы провели загрузку и подготовку данных, избавились от лишних столбцов и скважин-дубликатов. Далее мы обучили модель линейной регрессии на всех трёх регионах. Затем мы провели подготовку к расчету прибыли, выяснив необходимый объем сырья и написав функцию. 
Далее для того, чтобы рассчитать риски,  мы провели процедуру bootstrap (предварительно написав функцию).
После расчетов выяснилось, что лучше всего для разработки подходит регион номер 2, поэтому его мы выбрать и рекомендуем. 